In [104]:
import pandas as pd
import sqlalchemy as db

data = pd.read_csv('D:/dataset.csv', encoding='ISO-8859-1')
data.head()

C:\Users\Honor\AppData\Local\Temp\ipykernel_10296\2397117155.py:4: DtypeWarning:

Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.



,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [57]:
engine = db.create_engine('sqlite:///mydb.db')
conn = engine.connect()
metadata = db.MetaData()

In [3]:
location = db.Table(
    'Locations', metadata,
    db.Column('location_id', db.Integer,  primary_key=True),
    db.Column('country', db.Integer),
    db.Column('country_txt', db.Text),
    db.Column('region', db.Integer),
    db.Column('region_txt', db.Text),
    db.Column('provstate', db.Text),
    db.Column('city', db.Text),
)

In [4]:
attack_type = db.Table(
    'Attack types', metadata,
    db.Column('type_id', db.Integer, primary_key=True),
    db.Column('attacktype1', db.Integer),
    db.Column('attacktype1_txt', db.Text),
    extend_existing=True
)

In [5]:
target_type = db.Table(
    'Target types', metadata,
    db.Column('target_id', db.Integer, primary_key=True),
    db.Column('targtype1', db.Integer),
    db.Column('targtype1_txt', db.Text),
    db.Column('targsubtype1', db.Integer),
    db.Column('targsubtype1_txt', db.Text),
    db.Column('corp1', db.Text),
    db.Column('target1', db.Text),
    extend_existing=True
)

In [6]:
weap_type = db.Table(
    'Weap types', metadata,
    db.Column('weap_id', db.Integer, primary_key=True),
    db.Column('weaptype1', db.Integer),
    db.Column('weaptype1_txt', db.Text),
    db.Column('weapsubtype1', db.Integer),
    db.Column('weapsubtype1_txt', db.Text),
    db.Column('weapdetail', db.Text)
)

In [7]:
prop_type = db.Table(
    'Prop', metadata,
    db.Column('prop_id', db.Integer, primary_key=True),
    db.Column('property', db.Integer),
    db.Column('propextent', db.Integer),
    db.Column('propextent_txt', db.Text),
    db.Column('propcomment', db.Text),
    extend_existing=True
)

In [8]:
notes = db.Table(
    'Notes', metadata,
    db.Column('note_id', db.Integer, primary_key=True),
    db.Column('addnotes',  db.Text),
    db.Column('scite1',  db.Text),
    db.Column('scite2',  db.Text),
    db.Column('scite3',  db.Text)
)

In [9]:
new_dataset = db.Table(
    'Dataset', metadata,
    db.Column('eventid', db.Integer, primary_key=True),
    db.Column('location', db.ForeignKey("Locations.location_id")),
    db.Column('attack', db.ForeignKey("Attack types.type_id")),
    db.Column('target', db.ForeignKey("Target types.target_id")),
    db.Column('weap types', db.ForeignKey("Weap types.weap_id")),
    db.Column('target type', db.ForeignKey("Prop.prop_id")),
    db.Column('notes', db.ForeignKey("Notes.note_id")),
    
)

In [10]:
metadata.create_all(engine)

In [7]:
columns = location.columns.keys()
print(columns)

['location_id', 'country', 'country_txt', 'region', 'region_txt', 'provstate', 'city']


Location

In [60]:
new_d = data[['country', 'country_txt', 'region', 'region_txt',
       'provstate', 'city']].drop_duplicates()
new_d.shape

(41577, 6)

In [61]:
uni_data = new_d.values.tolist()
uni_data

[[58,
  'Dominican Republic',
  2,
  'Central America & Caribbean',
  nan,
  'Santo Domingo'],
 [130, 'Mexico', 1, 'North America', 'Federal', 'Mexico city'],
 [160, 'Philippines', 5, 'Southeast Asia', 'Tarlac', 'Unknown'],
 [78, 'Greece', 8, 'Western Europe', 'Attica', 'Athens'],
 [101, 'Japan', 4, 'East Asia', 'Fukouka', 'Fukouka'],
 [217, 'United States', 1, 'North America', 'Illinois', 'Cairo'],
 [218, 'Uruguay', 3, 'South America', 'Montevideo', 'Montevideo'],
 [217, 'United States', 1, 'North America', 'California', 'Oakland'],
 [217, 'United States', 1, 'North America', 'Wisconsin', 'Madison'],
 [217, 'United States', 1, 'North America', 'Wisconsin', 'Baraboo'],
 [217, 'United States', 1, 'North America', 'Colorado', 'Denver'],
 [98, 'Italy', 8, 'Western Europe', 'Lazio', 'Rome'],
 [217, 'United States', 1, 'North America', 'Michigan', 'Detroit'],
 [217, 'United States', 1, 'North America', 'Puerto Rico', 'Rio Piedras'],
 [499, 'East Germany (GDR)', 9, 'Eastern Europe', 'Berlin'

In [62]:
step = 0
for k in range(1, len(uni_data), 1):
    insertion = location.insert().values([
                {'location_id': i, 'country': uni_data[i][0], 'country_txt': uni_data[i][1], 'region': uni_data[i][2],'region_txt': uni_data[i][3],
            'provstate': uni_data[i][4], 'city': uni_data[i][5]} for i in range(step, k)
    ])
    conn.execute(insertion)
    step = k
    


In [69]:
conn.execute(db.select(location)).fetchall()

[(0, 58, 'Dominican Republic', 2, 'Central America & Caribbean', None, 'Santo Domingo'),
 (1, 130, 'Mexico', 1, 'North America', 'Federal', 'Mexico city'),
 (2, 160, 'Philippines', 5, 'Southeast Asia', 'Tarlac', 'Unknown'),
 (3, 78, 'Greece', 8, 'Western Europe', 'Attica', 'Athens'),
 (4, 101, 'Japan', 4, 'East Asia', 'Fukouka', 'Fukouka'),
 (5, 217, 'United States', 1, 'North America', 'Illinois', 'Cairo'),
 (6, 218, 'Uruguay', 3, 'South America', 'Montevideo', 'Montevideo'),
 (7, 217, 'United States', 1, 'North America', 'California', 'Oakland'),
 (8, 217, 'United States', 1, 'North America', 'Wisconsin', 'Madison'),
 (9, 217, 'United States', 1, 'North America', 'Wisconsin', 'Baraboo'),
 (10, 217, 'United States', 1, 'North America', 'Colorado', 'Denver'),
 (11, 98, 'Italy', 8, 'Western Europe', 'Lazio', 'Rome'),
 (12, 217, 'United States', 1, 'North America', 'Michigan', 'Detroit'),
 (13, 217, 'United States', 1, 'North America', 'Puerto Rico', 'Rio Piedras'),
 (14, 499, 'East Germ

Attack type

In [19]:
attack_type_uni= data[['attacktype1', 'attacktype1_txt']].drop_duplicates()
attack_type_uni.shape

(9, 2)

In [20]:
data[['attacktype1', 'attacktype1_txt']].shape

(181691, 2)

In [21]:
attack_type_lst = attack_type_uni.values.tolist()
attack_type_lst

[[1, 'Assassination'],
 [6, 'Hostage Taking (Kidnapping)'],
 [3, 'Bombing/Explosion'],
 [7, 'Facility/Infrastructure Attack'],
 [2, 'Armed Assault'],
 [4, 'Hijacking'],
 [9, 'Unknown'],
 [8, 'Unarmed Assault'],
 [5, 'Hostage Taking (Barricade Incident)']]

In [28]:
step = 0
for p in range(1, len(attack_type_lst), 1):
    insertion = attack_type.insert().values([
                {'type_id': i, 'attacktype1': attack_type_lst[i][0], 
                 'attacktype1_txt': attack_type_lst[i][1]} for i in range(step, p)
    ])
    conn.execute(insertion)
    step = p

In [29]:
conn.execute(db.select(attack_type)).fetchall()

[(0, 1, 'Assassination'),
 (1, 6, 'Hostage Taking (Kidnapping)'),
 (2, 3, 'Bombing/Explosion'),
 (3, 7, 'Facility/Infrastructure Attack'),
 (4, 2, 'Armed Assault'),
 (5, 4, 'Hijacking'),
 (6, 9, 'Unknown'),
 (7, 8, 'Unarmed Assault')]

Target type

In [34]:
target_type_uni= data[['targtype1', 'targtype1_txt', 'targsubtype1', 
                      'targsubtype1_txt', 'corp1', 'target1']].drop_duplicates()
target_type_uni.shape

(108000, 6)

In [35]:
target_type_lst = target_type_uni.values.tolist()
target_type_lst

[[14,
  'Private Citizens & Property',
  68.0,
  'Named Civilian',
  nan,
  'Julio Guzman'],
 [7,
  'Government (Diplomatic)',
  45.0,
  'Diplomatic Personnel (outside of embassy, consulate)',
  'Belgian Ambassador Daughter',
  'Nadine Chaval, daughter'],
 [10,
  'Journalists & Media',
  54.0,
  'Radio Journalist/Staff/Facility',
  'Voice of America',
  'Employee'],
 [7,
  'Government (Diplomatic)',
  46.0,
  'Embassy/Consulate',
  nan,
  'U.S. Embassy'],
 [7,
  'Government (Diplomatic)',
  46.0,
  'Embassy/Consulate',
  nan,
  'U.S. Consulate'],
 [3,
  'Police',
  22.0,
  'Police Building (headquarters, station, school)',
  'Cairo Police Department',
  'Cairo Police Headquarters'],
 [3,
  'Police',
  25.0,
  'Police Security Forces/Officers',
  'Uruguayan Police',
  'Juan Maria de Lucah/Chief of Directorate of info. and intell.'],
 [21,
  'Utilities',
  107.0,
  'Electricity',
  'Pacific Gas & Electric Company',
  'Edes Substation'],
 [4,
  'Military',
  28.0,
  'Military Recruiting S

In [42]:
step = 0
for n in range(1, len(target_type_lst), 1):
    insertion = target_type.insert().values([
                {'target_id': i, 'targtype1': target_type_lst[i][0], 'targtype1_txt': target_type_lst[i][1], 
                'targsubtype1': target_type_lst[i][2], 'targsubtype1_txt': target_type_lst[i][3], 
                'corp1': target_type_lst[i][4], 'target1': target_type_lst[i][5]
                } for i in range(step, n)
    ])
    conn.execute(insertion)
    step = n

In [43]:
conn.execute(db.select(target_type)).fetchall()

[(0, 14, 'Private Citizens & Property', 68, 'Named Civilian', None, 'Julio Guzman'),
 (1, 7, 'Government (Diplomatic)', 45, 'Diplomatic Personnel (outside of embassy, consulate)', 'Belgian Ambassador Daughter', 'Nadine Chaval, daughter'),
 (2, 10, 'Journalists & Media', 54, 'Radio Journalist/Staff/Facility', 'Voice of America', 'Employee'),
 (3, 7, 'Government (Diplomatic)', 46, 'Embassy/Consulate', None, 'U.S. Embassy'),
 (4, 7, 'Government (Diplomatic)', 46, 'Embassy/Consulate', None, 'U.S. Consulate'),
 (5, 3, 'Police', 22, 'Police Building (headquarters, station, school)', 'Cairo Police Department', 'Cairo Police Headquarters'),
 (6, 3, 'Police', 25, 'Police Security Forces/Officers', 'Uruguayan Police', 'Juan Maria de Lucah/Chief of Directorate of info. and intell.'),
 (7, 21, 'Utilities', 107, 'Electricity', 'Pacific Gas & Electric Company', 'Edes Substation'),
 (8, 4, 'Military', 28, 'Military Recruiting Station/Academy', 'R.O.T.C.', 'R.O.T.C. offices at University of Wisconsin,

Weap 

In [77]:
weap_uni= data[['weaptype1', 'weaptype1_txt', 'weapsubtype1', 
                      'weapsubtype1_txt', 'weapdetail']].drop_duplicates()
weap_lst = weap_uni.values.tolist()

In [78]:
step = 0
for n in range(1, len(weap_lst), 1):
    insertion = weap_type.insert().values([
                {'weap_id': i, 'weaptype1': weap_lst[i][0], 'weaptype1_txt': weap_lst[i][1], 
                'weapsubtype1': weap_lst[i][2], 'weapsubtype1_txt': weap_lst[i][3], 
                'weapdetail': weap_lst[i][4]
                } for i in range(step, n)
    ])
    conn.execute(insertion)
    step = n

conn.execute(db.select(weap_type)).fetchall()

[(0, 13, 'Unknown', None, None, None),
 (1, 6, 'Explosives', 16, 'Unknown Explosive Type', 'Explosive'),
 (2, 8, 'Incendiary', None, None, 'Incendiary'),
 (3, 5, 'Firearms', 5, 'Unknown Gun Type', 'Several gunshots were fired.'),
 (4, 5, 'Firearms', 2, 'Automatic or Semi-Automatic Rifle', 'Automatic firearm'),
 (5, 6, 'Explosives', 16, 'Unknown Explosive Type', None),
 (6, 8, 'Incendiary', 19, 'Molotov Cocktail/Petrol Bomb', 'Firebomb consisting of gasoline'),
 (7, 8, 'Incendiary', 20, 'Gasoline or Alcohol', 'Poured gasoline on the floor and lit it with a match'),
 (8, 8, 'Incendiary', 19, 'Molotov Cocktail/Petrol Bomb', 'Molotov cocktail'),
 (9, 5, 'Firearms', 4, 'Rifle/Shotgun (non-automatic)', 'Rifle - carbine; Pistols'),
 (10, 8, 'Incendiary', 19, 'Molotov Cocktail/Petrol Bomb', 'Firebomb'),
 (11, 8, 'Incendiary', 18, 'Arson/Fire', 'Fire set in back of store'),
 (12, 6, 'Explosives', 31, 'Pipe Bomb', 'Crudely made pipe bomb.  Five inches long and a half inch thick filled with gunpo

PROP

In [89]:
prop_uni= data[['property', 'propextent', 'propextent_txt', 
                      'propcomment']].drop_duplicates()
prop_lst = prop_uni.values.tolist()

In [90]:
step = 0
for l in range(1, len(prop_lst), 1):
    insertion = prop_type.insert().values([
                {'prop_id': i, 'property': prop_lst[i][0], 'propextent': prop_lst[i][1], 
                'propextent_txt': prop_lst[i][2], 'propcomment': prop_lst[i][3]
                } for i in range(step, l)
    ])
    conn.execute(insertion)
    step = l

conn.execute(db.select(prop_type)).fetchall()

[(0, 14, 'Private Citizens & Property', '68.0', 'Named Civilian'),
 (1, 7, 'Government (Diplomatic)', '45.0', 'Diplomatic Personnel (outside of embassy, consulate)'),
 (2, 10, 'Journalists & Media', '54.0', 'Radio Journalist/Staff/Facility'),
 (3, 7, 'Government (Diplomatic)', '46.0', 'Embassy/Consulate'),
 (4, 7, 'Government (Diplomatic)', '46.0', 'Embassy/Consulate'),
 (5, 3, 'Police', '22.0', 'Police Building (headquarters, station, school)'),
 (6, 3, 'Police', '25.0', 'Police Security Forces/Officers'),
 (7, 21, 'Utilities', '107.0', 'Electricity'),
 (8, 4, 'Military', '28.0', 'Military Recruiting Station/Academy'),
 (9, 2, 'Government (General)', '21.0', 'Government Building/Facility/Office'),
 (10, 4, 'Military', '27.0', 'Military Barracks/Base/Headquarters/Checkpost'),
 (11, 4, 'Military', '28.0', 'Military Recruiting Station/Academy'),
 (12, 6, 'Airports & Aircraft', '42.0', 'Aircraft (not at an airport)'),
 (13, 2, 'Government (General)', '21.0', 'Government Building/Facility/

Notes

In [73]:
notes_uni= data[['addnotes', 'scite1', 'scite2', 'scite3']].drop_duplicates()
notes_lst = notes_uni.values.tolist()

In [74]:
step = 0
for m in range(1, len(notes_lst), 1):
    insertion = notes.insert().values([
                {'note_id': i, 'addnotes': notes_lst[i][0], 'scite1': notes_lst[i][1], 
                'scite2': notes_lst[i][2], 'scite3': notes_lst[i][3]
                } for i in range(step, m)
    ])
    conn.execute(insertion)
    step = m

conn.execute(db.select(notes)).fetchall()

[(0, None, None, None, None),
 (1, 'The Cairo Chief of Police, William Petersen, resigned as a result of the attack.', '"Police Chief Quits," Washington Post, January 2, 1970.', '"Cairo Police Chief Quits; Decries Local \'Militants\'," Afro-American, January 10, 1970.', 'Christopher Hewitt, "Political Violence and Terrorism in Modern America: A Chronology," Praeger Security International, 2005.'),
 (2, 'Damages were estimated to be between $20,000-$25,000.', 'Committee on Government Operations United States Senate, "Riots, Civil, and Criminal Disorders," U.S. Government Printing Office, August 6, 1970.', 'Christopher Hewitt, "Political Violence and Terrorism in Modern America: A Chronology," Praeger Security International, 2005.', None),
 (3, "The New Years Gang issue a communiqué to a local paper claiming that they perpetrated this attack.  The New Years Gang  previously attempted to fireb ... (319 characters truncated) ... trong would be captured after participating in the deadly bom

dataset

In [63]:
locations_data = pd.read_sql(location.select(), conn)
for row in range(0, data.shape[0], 10000):
    if data['city'][row] in locations_data['city'].values:
        location_id = int(locations_data.loc[locations_data['city'] == 
                                         data['city'][row], 'location_id'].values[0])
        insertion = new_dataset.insert().values(location = location_id)
        conn.execute(insertion)
        

In [64]:
conn.execute(db.select(new_dataset).limit(10)).fetchall()

[(1, 0, None, None, None, None, None),
 (2, 481, None, None, None, None, None),
 (3, 4243, None, None, None, None, None),
 (4, 2, None, None, None, None, None),
 (5, 4614, None, None, None, None, None),
 (6, 408, None, None, None, None, None),
 (7, 4381, None, None, None, None, None),
 (8, 6063, None, None, None, None, None),
 (9, 748, None, None, None, None, None),
 (10, 17074, None, None, None, None, None)]

In [56]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

try:
    session.query(new_dataset).delete()
    session.commit()
except Exception as e:
    session.rollback()
    raise
finally:
    session.close()


In [49]:
code = data['eventid'].drop_duplicates().tolist()
step = 0
for j in range(1, len(code), 1):
    insertion = new_dataset.insert().values([
                {'eventid': code[i]
                } 
                 for i in range(step, j)
    ])
    conn.execute(insertion)
    step = j

join

In [71]:
select_all_query = db.select(new_dataset.c.eventid, 
                              location.c.country,
                              location.c.country_txt,
                              location.c.region,
                              location.c.region_txt,
                              location.c.provstate,
                              location.c.city
                            ).join(
                                    location, new_dataset.c.location == location.c.location_id
                            )
select_result = conn.execute(select_all_query)
select_result.fetchall()

[(1, 58, 'Dominican Republic', 2, 'Central America & Caribbean', None, 'Santo Domingo'),
 (2, 603, 'United Kingdom', 8, 'Western Europe', 'Northern Ireland', 'Aughnacloy'),
 (3, 159, 'Peru', 3, 'South America', 'Huancavelica', 'chontacancha'),
 (4, 160, 'Philippines', 5, 'Southeast Asia', 'Tarlac', 'Unknown'),
 (5, 155, 'West Bank and Gaza Strip', 10, 'Middle East & North Africa', 'Gaza Strip', 'Khan Yunis'),
 (6, 603, 'United Kingdom', 8, 'Western Europe', 'Northern Ireland', 'Dungannon'),
 (7, 110, 'Lebanon', 10, 'Middle East & North Africa', 'South', 'Jezzine'),
 (8, 45, 'Colombia', 3, 'South America', 'Santander', 'Bucaramanga district'),
 (9, 185, 'Spain', 8, 'Western Europe', 'Alava', 'Vitoria'),
 (10, 167, 'Russia', 9, 'Eastern Europe', 'Ingushetia (Republic)', 'Malgobek'),
 (11, 160, 'Philippines', 5, 'Southeast Asia', 'Unknown', 'Laoag'),
 (12, 160, 'Philippines', 5, 'Southeast Asia', 'North Cotabato', 'Midsayap'),
 (13, 95, 'Iraq', 10, 'Middle East & North Africa', 'Nineveh',

In [ ]:
join(
                                    attack_type, new_dataset.c.attack == attack_type.c.type_id
                            ).join(
                                    target_type, new_dataset.c.target == target_type.c.target_id
                            ).join(
                                    weap_type, new_dataset.c.weap_types == weap_type.c.weap_id
                            ).join(
                                    prop_type, new_dataset.c.target_type == prop_type.c.prop_id
                            ).join(
                                    notes, new_dataset.c.notes == note.c.note_id
                            )


Анализ данных 

In [86]:
import plotly


import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'notebook' 
   


In [88]:
df = data[data.iyear >= 2000].iloc[:100,:]
pie1 = df.groupby('region_txt')['region_txt'].count()
labels = df['region_txt'].unique()


fig = {
  "data": [
    {
      "values": pie1.tolist(),
      "labels": labels.tolist(),
      "domain": {"x": [0, .5]},
      "name": "Counties",
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Terrorist attacks after 2000",
        "annotations": [
            { "font": { "size": 20},
              "showarrow": False,
              "text": "Number of Attacks",
                "x": 0.20,
                "y": 1
            },
        ]
    }
}
fig = go.Figure(fig)
fig.show()

In [112]:
df = data.iloc[:100,:]


business_data = df[df.targtype1_txt == 'Business']
military_data = df[df.targtype1_txt == 'Military']

years = df['iyear']

business_count = business_data.groupby('iyear').size().reindex(years, fill_value=0)
military_count = military_data.groupby('iyear').size().reindex(years, fill_value=0)

trace1 = go.Bar(
    x=years,
    y=business_count,
    name="Business",
    marker=dict(color='rgba(171, 50, 96, 0.6)')
)

trace2 = go.Bar(
    x=years,
    y=military_count,
    name="Military",
    marker=dict(color='rgba(12, 50, 196, 0.6)')
)

data_ = [trace1, trace2]
layout = go.Layout(
    barmode='overlay',
    title='Number of incidents by year for Business and Military purposes',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Count'),
)

fig = go.Figure(data=data, layout=layout)
fig.show()


AttributeError: 'list' object has no attribute 'iloc'

In [102]:
data['targtype1_txt']

TypeError: list indices must be integers or slices, not str